# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_test').getOrCreate() 

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
df = spark.read.csv('cruise_ship_info.csv', header=True, inferSchema=True)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
df.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



# Dealing with CATEGORICAL VARIABLES

In [7]:
# Shipnames might not be relevant
df.groupby('Ship_name').count().show()

+------------+-----+
|   Ship_name|count|
+------------+-----+
|       Virgo|    1|
|     Fortuna|    1|
|      Shadow|    1|
|     Empress|    1|
|        Wind|    2|
|    Paradise|    1|
|        Surf|    1|
|      Wonder|    1|
|       Magic|    1|
|    Symphony|    1|
|    Sinfonia|    1|
| Inspiration|    1|
|   Millenium|    1|
|    Solstice|    1|
|PrideofAloha|    1|
|     Majesty|    2|
|     Ventura|    1|
|   Romantica|    1|
|      Spirit|    4|
|       Oasis|    1|
+------------+-----+
only showing top 20 rows



In [8]:
df.agg({'Ship_name':'count'}).show()

+----------------+
|count(Ship_name)|
+----------------+
|             158|
+----------------+



In [9]:
from pyspark.sql.functions import col, countDistinct

In [1]:
#df.agg(countDistinct(col('Ship_name')).alias('Ship_name'),
#       countDistinct(col('Cruise_line')).alias('Cruise_line')).show()

In [11]:
df.agg(countDistinct(df['Ship_name']).alias('Ship_name'),
       countDistinct(df['Cruise_line']).alias('Cruise_line')).show()

+---------+-----------+
|Ship_name|Cruise_line|
+---------+-----------+
|      138|         20|
+---------+-----------+



In [12]:
df.agg(*(countDistinct(df[c]).alias(c) for c in df.columns)).show()

+---------+-----------+---+-------+----------+------+------+-----------------+----+
|Ship_name|Cruise_line|Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+-------+----------+------+------+-----------------+----+
|      138|         20| 31|     94|       104|    80|    98|              109|  91|
+---------+-----------+---+-------+----------+------+------+-----------------+----+



In [13]:
# Cruise_lline could be essential # Only 20  cruisne_line 
# Thus we can use categorical encoding
df.groupby('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [14]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_numerical')
df2 = indexer.fit(df).transform(df)
df2.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_numerical=16.0)

In [15]:
# Right way would be using
# 1. dummy variable conversion
# 2. one hot encoding for the categorical featureS

## Data transformation

In [16]:
# Import VectorAssembler and Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [17]:
df2.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_numerical']

In [18]:
# Let's start with numeric column only

In [19]:
assembler = VectorAssembler(inputCols=[ 'Age','Tonnage','passengers','length','cabins',
                                       'passenger_density', 'cruise_numerical'],
                            outputCol='features')

In [20]:
output = assembler.transform(df2)
output.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_numerical=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [21]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_numerical: double (nullable = false)
 |-- features: vector (nullable = true)



In [22]:
# we got all numeric column values of each row as a vector

In [23]:
data = output.select(output['features'], output['crew'])
data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [24]:
# DIVIDE DATA INTO TRAIN AND TEST SET

In [25]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [26]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              111|
|   mean|7.620360360360371|
| stddev| 3.46117292991524|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [27]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                47|
|   mean|  8.20468085106383|
| stddev|3.6056747852238784|
|    min|               0.6|
|    max|              19.1|
+-------+------------------+



In [28]:
lr = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='prediction')

In [29]:
lr_model = lr.fit(train_data)

In [30]:
lr_model.coefficients

DenseVector([-0.0174, 0.0119, -0.1305, 0.3924, 0.7855, -0.0044, 0.0611])

In [31]:
lr_model.intercept

-0.6931002203796638

In [32]:
lr_model.summary.r2

0.9588306329012881

In [33]:
lr_model.summary.rootMeanSquaredError

0.6991095695669541

In [44]:
lr_model.summary.predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[4.0,220.0,54.0,1...| 21.0| 20.47599182052267|
|[5.0,86.0,21.04,9...|  8.0| 9.202621606249394|
|[5.0,115.0,35.74,...| 12.2| 11.89897167301658|
|[5.0,122.0,28.5,1...|  6.7| 6.617274564230313|
|[5.0,133.5,39.59,...|13.13|13.069610280892784|
|[5.0,160.0,36.34,...| 13.6|14.824383308406404|
|[6.0,30.276999999...| 3.55| 4.568391848604271|
|[6.0,93.0,23.94,9...|11.09|10.448501286536514|
|[6.0,110.23899999...| 11.5|11.028413251694955|
|[6.0,112.0,38.0,9...| 10.9|11.266963118268281|
|[6.0,158.0,43.7,1...| 13.6|13.774563207134616|
|[7.0,89.6,25.5,9....| 9.87|10.983789326899133|
|[8.0,77.499,19.5,...|  9.0|  8.51109979649432|
|[9.0,85.0,19.68,9...| 8.69| 9.353086216687831|
|[9.0,90.09,25.01,...| 8.69| 9.169062145880325|
|[9.0,105.0,27.2,8...|10.68|11.130671949115706|
|[9.0,110.0,29.74,...| 11.6|11.893417550686108|
|[9.0,113.0,26.74,...|12.38|11.176508711

In [34]:
test_results = lr_model.evaluate(test_data)

In [35]:
test_results.r2

0.8502379199410085

In [45]:
test_results.rootMeanSquaredError

1.3804397977854062

In [47]:
test_results.predictions.show()

+--------------------+-----+------------------+
|            features| crew|        prediction|
+--------------------+-----+------------------+
|[6.0,30.276999999...| 3.55| 4.568391848604271|
|[6.0,90.0,20.0,9....|  9.0| 10.24913716900857|
|[6.0,113.0,37.82,...| 12.0|   11.564868716989|
|[7.0,116.0,31.0,9...| 12.0|12.440292520030033|
|[7.0,158.0,43.7,1...| 13.6|13.706194041295388|
|[8.0,91.0,22.44,9...| 11.0|  9.98916460406095|
|[8.0,110.0,29.74,...| 11.6|11.906856524132332|
|[9.0,59.058,17.0,...|  7.4| 7.581434716832957|
|[9.0,81.0,21.44,9...| 10.0| 9.430063142583707|
|[9.0,88.5,21.24,9...| 10.3| 9.431165381398502|
|[10.0,58.825,15.6...|  7.0|7.2987382791673365|
|[10.0,81.76899999...| 8.42| 8.705223401635767|
|[10.0,105.0,27.2,...|10.68|11.113309390498555|
|[10.0,151.4,26.2,...|12.53|11.528953324745848|
|[11.0,85.0,18.48,...|  8.0| 8.687254986175441|
|[11.0,86.0,21.24,...|  9.3| 9.371861052610337|
|[11.0,110.0,29.74...| 19.1|11.870471379165032|
|[12.0,2.329,0.94,...|  0.6|0.7770207666

In [37]:
# Make up unseen data

In [38]:
unlabeled_data = test_data.select(test_data['features'])
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,90.0,20.0,9....|
|[6.0,113.0,37.82,...|
|[7.0,116.0,31.0,9...|
|[7.0,158.0,43.7,1...|
|[8.0,91.0,22.44,9...|
|[8.0,110.0,29.74,...|
|[9.0,59.058,17.0,...|
|[9.0,81.0,21.44,9...|
|[9.0,88.5,21.24,9...|
|[10.0,58.825,15.6...|
|[10.0,81.76899999...|
|[10.0,105.0,27.2,...|
|[10.0,151.4,26.2,...|
|[11.0,85.0,18.48,...|
|[11.0,86.0,21.24,...|
|[11.0,110.0,29.74...|
|[12.0,2.329,0.94,...|
|[12.0,58.6,15.66,...|
|[12.0,88.5,21.24,...|
+--------------------+
only showing top 20 rows



In [39]:
pred = lr_model.transform(unlabeled_data)
pred.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...| 4.568391848604271|
|[6.0,90.0,20.0,9....| 10.24913716900857|
|[6.0,113.0,37.82,...|   11.564868716989|
|[7.0,116.0,31.0,9...|12.440292520030033|
|[7.0,158.0,43.7,1...|13.706194041295388|
|[8.0,91.0,22.44,9...|  9.98916460406095|
|[8.0,110.0,29.74,...|11.906856524132332|
|[9.0,59.058,17.0,...| 7.581434716832957|
|[9.0,81.0,21.44,9...| 9.430063142583707|
|[9.0,88.5,21.24,9...| 9.431165381398502|
|[10.0,58.825,15.6...|7.2987382791673365|
|[10.0,81.76899999...| 8.705223401635767|
|[10.0,105.0,27.2,...|11.113309390498555|
|[10.0,151.4,26.2,...|11.528953324745848|
|[11.0,85.0,18.48,...| 8.687254986175441|
|[11.0,86.0,21.24,...| 9.371861052610337|
|[11.0,110.0,29.74...|11.870471379165032|
|[12.0,2.329,0.94,...|0.7770207666278356|
|[12.0,58.6,15.66,...| 7.400094071512967|
|[12.0,88.5,21.24,...| 9.331945542293278|
+--------------------+------------

If we see the R2 with test data is high and other errors are low then there is a high chance that the target `crew` in this case could be strongly correlated with one of the feature.

Let's test that

In [40]:
from pyspark.sql.functions import corr

In [41]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [42]:
df.agg(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [43]:
df.agg(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [197]:
# These 2 columns i.e. cabins and passengers are highly correlated to the target i.e. `crew`